## Cortical Microcircuit Modeling: Optogenetic Perturbations and Network Stability Analysis 
### (Aug. 2025) - updated

### <span style='color:#336C43; font-weight:bold'> Overview: This project investigates the dynamic properties of a biologically-inspired cortical microcircuit using the NetPyNE simulation framework. The study focuses on understanding gain control mechanisms and testing for Inhibition-Stabilized Network (ISN) properties through targeted optogenetic manipulations.</span>

### <span style='color:#2E86C1; font-weight:bold'> Features: 
* *NEURON-based neural simulation via NetPyNE framework*  
* *Biophysical modeling of Hodgkin-Huxley neuron dynamics with realistic membrane properties*  
* *Probabilistic synaptic connectivity with different weight parameters for excitatory and inhibitory connections*  
* *Optogenetic perturbation simulation using current clamp protocols*  
* *Spike data analysis and firing rate calculation with custom Python algorithms*
* *Population dynamics visualization through raster plots, PSTHs, and bar charts*
* *Network stability assessment through paradoxical response testing*  
* *Data-driven parameterization from experimental electrophysiology data*
    



In [7]:
from netpyne import specs, sim
from neuron import h

# NEURON libraries for 3D morphology
h.load_file('stdrun.hoc')
h.load_file('import3d.hoc')

netParams = specs.NetParams()

# netParams general settings -- define the size of the network
netParams.sizeX = 500 
netParams.sizeY = 1000 
netParams.sizeZ = 500



#  PYRAMIDAL CELL (Exc):
netParams.cellParams['PYR'] = {
    'secs': {
        'soma': {
            'geom': {
                'diam': 18.8, 
                'L': 18.8, 
                'Ra': 123.0
            },
            'mechs': {
                'hh': {
                    'gnabar': 0.12, 
                    'gkbar': 0.036, 
                    'gl': 0.001, 
                    'el': -70
                }
            }
        },
        'dend': {
            'geom': {
                'diam': 2.0, 
                'L': 150.0, 
                'Ra': 150.0, 
                'cm': 1.0
            },
            'topol': {
                'parentSec': 'soma', 
                'parentX': 1.0, 
                'childX': 0
            },
            'mechs': {
                'pas': {
                    'g': 0.0002, 
                    'e': -70
                }
            }
        }
    }
}

# INTERNEURON: Fast-spiking properties with higher sodium/potassium conductance
netParams.cellParams['INH'] = {
    'secs': {
        'soma': {
            'geom': {
                'diam': 16.0, 
                'L': 16.0, 
                'Ra': 125.0
            },
            'mechs': {
                'hh': {
                    'gnabar': 0.14, 
                    'gkbar': 0.045, 
                    'gl': 0.00095, 
                    'el': -70
                }
            }
        }
    }
}

# populations
netParams.popParams['Exc_L23'] = {'numCells': 40, 'cellType': 'PYR', 'cellModel': 'HH', 'ynormRange': [0.2, 0.6]}
netParams.popParams['Exc_L5']  = {'numCells': 40, 'cellType': 'PYR', 'cellModel': 'HH', 'ynormRange': [0.5, 0.8]}
netParams.popParams['Inh_PV']  = {'numCells': 10, 'cellType': 'INH', 'cellModel': 'HH', 'ynormRange': [0.4, 0.9]}
netParams.popParams['Inh_SOM'] = {'numCells': 10, 'cellType': 'INH', 'cellModel': 'HH', 'ynormRange': [0.6, 0.9]}

# synaptic mechanisms & connectivity
netParams.synMechParams['exc'] = {'mod': 'Exp2Syn', 'tau1': 0.1, 'tau2': 1.0, 'e': 0}
netParams.synMechParams['inh'] = {'mod': 'Exp2Syn', 'tau1': 0.5, 'tau2': 5.0, 'e': -80}

# if you're trying to import SWC morphology files:
# SWC imports usually name sections 'dend_0', 'dend_1', etc.
# NetPyNE allows using 'dend' to match all sections containing that string.

dend_target = 'dend' 

netParams.connParams['PV->Exc'] = {
    'preConds': {'pop': 'Inh_PV'}, 
    'postConds': {'pop': ['Exc_L5', 'Exc_L23']},
    'sec': 'soma', 
    'loc': 0.5,
    'probability': 0.1, 
    'weight': 0.005,
    'synMech': 'inh', 
    'delay': 2.0
}



netParams.connParams['Exc->PV'] = {
    'preConds': {'pop': ['Exc_L5', 'Exc_L23']}, 
    'postConds': {'pop': 'Inh_PV'},
    'sec': 'soma', 
    'loc': 0.5,
    'probability': 0.5, 
    'weight': 0.004,
    'synMech': 'exc', 
    'delay': 2.0
}


netParams.connParams['SOM->All'] = {
    'preConds': {'pop': 'Inh_SOM'}, 
    'postConds': {'pop': ['Exc_L5', 'Exc_L23', 'Inh_PV']},
    'sec': dend_target, 
    'loc': 0.5,
    'probability': 0.2, 
    'weight': 0.0006,
    'synMech': 'inh', 
    'delay': 2.0
}

netParams.connParams['Exc->Exc'] = {
    'preConds': {'pop': ['Exc_L5', 'Exc_L23']}, 
    'postConds': {'pop': ['Exc_L5', 'Exc_L23', 'Inh_PV', 'Inh_SOM']},
    'sec': dend_target, 
    'loc': 0.5,
    'probability': 0.5, 
    'weight': 0.0005,
    'synMech': 'exc', 
    'delay': 3.0
}

netParams.stimSourceParams['bkg'] = {'type': 'NetStim', 'rate': 40, 'noise': 0.8}
netParams.stimTargetParams['bkg->all'] = {
    'source': 'bkg', 
    'conds': {'pop': ['Exc_L5', 'Exc_L23', 'Inh_PV', 'Inh_SOM']}, 
    'sec': 'soma',
    'loc': 0.5,
    'weight': 0.007, 
    'delay': 2.0, 
    'synMech': 'exc'
}

# simu config
simConfig = specs.SimConfig()
simConfig.duration = 1000
simConfig.dt = 0.05
simConfig.filename = r'results\microcircuit_81_results'
simConfig.saveJson = True
simConfig.saveDataInclude = ['simData', 'netParams', 'net']

# recording
simConfig.recordCells = [0, 45, 80, 95] 
simConfig.recordTraces = {
    'V_soma': {
        'sec': 'soma',
        'loc': 0.5, 
        'var': 'v'
    }
}

# analysis and plotting
pop_colors = {
    'Exc_L23': "#E74C3C", 
    'Exc_L5': '#EB984E', 
    'Inh_PV': "#2E86C1", 
    'Inh_SOM': '#28B463'
}

simConfig.analysis['plotRaster'] = {
    'saveFig': True, 
    'popColors': pop_colors, 
    'orderInverse': True, 
    'spikeHist': 'bin',
    'showFig': True
}

simConfig.analysis['plotTraces'] = {
    'include': [0, 45, 80, 95], 
    'saveFig': True,
    'showFig': True
}

simConfig.analysis['plotConn'] = {'saveFig': True, 'showFig': True}

simConfig.analysis['plot2Dnet'] = {
    'saveFig': True,
    'showFig': True,
    'figSize': (10, 10),
    'view': 'xy',
    'showConns': True,
    'popColors': pop_colors,
    'fontSize': 8
}

simConfig.analysis['plotRatePSD'] = {
    'include': ['allCells'],
    'timeRange': [100, 1000],
    'binSize': 5,
    'maxFreq': 100,
    'minFreq': 0, 
    'transformMethod': 'fft', 
    'smooth': 10,
    'saveFig': True,
    'showFig': True
}

simConfig.analysis['plotSpikeStats'] = {
    'saveFig': True,
    'showFig': True,
    'timeRange': [0, 1000]
}

# run sim
sim.create(netParams=netParams, simConfig=simConfig)
sim.simulate()
sim.analyze()

# export spikes in csv
if sim.rank == 0:
    import csv
    with open(simConfig.filename + '_spikes.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['spkt', 'spkid'])
        writer.writerows(zip(sim.allSimData['spkt'], sim.allSimData['spkid']))
    print(f"DONE: Data saved to {simConfig.filename}")


Start time:  2026-02-01 13:18:08.475768




Creating network of 4 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 100 
  Done; cell creation time = 0.03 s.
Making connections...


  PV->Exc: 100%|##########| Creating synaptic connections for 80/80 postsynaptic cells on node 0 (probabilistic connectivity)
  Exc->PV: 100%|##########| Creating synaptic connections for 10/10 postsynaptic cells on node 0 (probabilistic connectivity)
  SOM->All: 100%|##########| Creating synaptic connections for 90/90 postsynaptic cells on node 0 (probabilistic connectivity)
  Exc->Exc: 100%|##########| Creating synaptic connections for 100/100 postsynaptic cells on node 0 (probabilistic connectivity)


  Number of connections on node 0: 4359 
  Number of synaptic contacts on node 0: 4533 
  Done; cell connection time = 0.59 s.
Adding stims...
  Number of stims on node 0: 100 
  Done; cell stims creation time = 0.01 s.
Recording 4 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 13.15 s; real-time ratio: 0.08.

Gathering data...
  Done; gather time = 1.47 s.

Analyzing...
  Cells: 100
  Connections: 4459 (44.59 per cell)
  Synaptic contacts: 4633 (46.33 per cell)
  Spikes: 2053 (20.53 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 13.15 s
Saving output as results\microcircuit_81_results_data.json ... 
Finished saving!
  Done; saving time = 0.29 s.
Preparing spike data...
Plotting raster...
Plotting recorded cell traces ... cell
Plotting connectivity matrix...
Plotting 2D representation of network cell locations and connections...
Plotting firing rate power spectral density (PSD) ...
Plotting spike stats...
  Done; plotting time = 3.6

## 3D Visualization of Network

In [ ]:
from neuron import gui
import plotly.graph_objects as go

# extract cell positions
x = [cell.tags['x'] for cell in sim.net.cells]
y = [cell.tags['y'] for cell in sim.net.cells]
z = [cell.tags['z'] for cell in sim.net.cells]
colors = [pop_colors[cell.tags['pop']] for cell in sim.net.cells]

fig = go.Figure(data=[go.Scatter3d(
    x=x, y=y, z=z,
    mode='markers',
    marker=dict(
        size=10,
        color=colors,
        opacity=0.8,
        symbol='circle'
    )
)])

fig.update_layout(title="Microcircuit 3D View")
fig.show()

### Optogenetic Simulation

In [15]:
sim.initialize() 

# laser definition
netParams.stimSourceParams['optogenetic_laser'] = {
    'type': 'IClamp',
    'delay': 400,    
    'dur': 200,      
    'amp': -0.75     
}

# targeted target :D
netParams.stimTargetParams['laser->PV_cells'] = {
    'source': 'optogenetic_laser',
    'conds': {'pop': 'Inh_PV'},  
    'sec': 'soma',
    'loc': 0.5
}

simConfig.saveDataInclude = ['simData', 'netParams', 'net']
simConfig.filename = r'results\optogenetic_experiment_results'

# define exactly what plots we want 
simConfig.analysis['plotRaster'] = {
    'include': ['allCells'], 
    'saveFig': True,
    'timeRange': [0, 1000], 
    'lines': [400, 600], #  matches 'dur': 200
    'showFig': False,
    'popColors': pop_colors
}

simConfig.analysis['plotTraces'] = {
    'include': [('Inh_PV', 0)], # watch the laser hit one PV cell
    'saveFig': True,
    'showFig': False
}

#  run the simulation
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)


Start time:  2026-02-01 13:23:50.044971

Start time:  2026-02-01 13:23:50.066981




Creating network of 4 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 100 
  Done; cell creation time = 0.03 s.
Making connections...


  PV->Exc: 100%|##########| Creating synaptic connections for 80/80 postsynaptic cells on node 0 (probabilistic connectivity)
  Exc->PV: 100%|##########| Creating synaptic connections for 10/10 postsynaptic cells on node 0 (probabilistic connectivity)
  SOM->All: 100%|##########| Creating synaptic connections for 90/90 postsynaptic cells on node 0 (probabilistic connectivity)
  Exc->Exc: 100%|##########| Creating synaptic connections for 100/100 postsynaptic cells on node 0 (probabilistic connectivity)


  Number of connections on node 0: 4359 
  Number of synaptic contacts on node 0: 4533 
  Done; cell connection time = 0.60 s.
Adding stims...
  Number of stims on node 0: 120 
  Done; cell stims creation time = 0.01 s.
Recording 4 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 12.54 s; real-time ratio: 0.08.

Gathering data...
  Done; gather time = 0.13 s.

Analyzing...
  Cells: 100
  Connections: 4459 (44.59 per cell)
  Synaptic contacts: 4643 (46.43 per cell)
  Spikes: 2152 (21.52 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 12.54 s
Saving output as results\optogenetic_experiment_results_data.json ... 
Finished saving!
  Done; saving time = 0.27 s.
Preparing spike data...
Plotting raster...
Plotting recorded cell traces ... cell
Plotting connectivity matrix...
Plotting 2D representation of network cell locations and connections...
Plotting firing rate power spectral density (PSD) ...
Plotting spike stats...
  Done; plotting tim

In [28]:
import os
import numpy as np
import matplotlib.pyplot as plt

# extract raw data
spkts = np.array(sim.allSimData['spkt'])
spkids = np.array(sim.allSimData['spkid'])

#  define windows
pre_laser = [200, 400]   # baseline
during_laser = [400, 600] # perturbation

 # gid 
exc_gids = [c.gid for c in sim.net.cells if c.tags['pop'] in ['Exc_L5', 'Exc_L23']]
pv_gids = [c.gid for c in sim.net.cells if c.tags['pop'] == 'Inh_PV']

# rate calculation
def get_pop_rate(gids, time_range):
    if not gids: return 0
    # filter spikes by time window
    mask = (spkts >= time_range[0]) & (spkts < time_range[1])
    spikes_in_window = spkids[mask]
    
    # count how many of those spikes belong to the GID list
    count = np.isin(spikes_in_window, gids).sum()
    
    duration_sec = (time_range[1] - time_range[0]) / 1000.0
    return count / (len(gids) * duration_sec)



rate_control = get_pop_rate(exc_gids, pre_laser)
rate_silenced = get_pop_rate(exc_gids, during_laser)


if rate_control > 0:
    gain_change = ((rate_silenced - rate_control) / rate_control) * 100
    print(f"Gain Increase: {gain_change:.1f}%")
else:
    print("Baseline activity too low to calculate gain.")
    import matplotlib.pyplot as plt
import numpy as np

# bar Chart
conditions = ['Control', 'PV Silenced']
rates = [rate_control, rate_silenced]
colors = ['#E74C3C', "#4D0C06"]


plt.figure(figsize=(12, 5))

# plot 1; bar chart
plt.subplot(1, 2, 1)
bars = plt.bar(conditions, rates, color=colors, edgecolor='black', alpha=0.8)
plt.ylabel('Excitatory Firing Rate (Hz)', fontsize=12)
plt.title('Disinhibition Effect (Gain Change)', fontsize=11, fontweight='bold')
plt.ylim(0, max(rates) * 1.3)

# value labels 
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.2, f'{yval:.2f} Hz', ha='center', fontweight='bold')

# plot 2; PSTH
plt.subplot(1, 2, 2)

# bin the spikes into 10ms windows 
bin_size = 10 
bins = np.arange(0, 1000 + bin_size, bin_size)
counts, _ = np.histogram(spkts[np.isin(spkids, exc_gids)], bins=bins)

# convert counts to Hz
num_exc = len(exc_gids)
rate_over_time = (counts / num_exc) / (bin_size / 1000.0)
bin_centers = bins[:-1] + bin_size/2

plt.plot(bin_centers, rate_over_time, color='#E74C3C', lw=2, label='Exc Population Rate')
plt.xlim(200, 1000)

# shade the "laser on" window 
plt.axvspan(400, 600, color='blue', alpha=0.15, label='PV Silencing')
plt.axvline(400, color='blue', linestyle='--', alpha=0.5)
plt.axvline(600, color='blue', linestyle='--', alpha=0.5)

plt.xlabel('Time (ms)', fontsize=10)
plt.ylabel('Firing Rate (Hz)', fontsize=10)
plt.title('Population Dynamics during Perturbation', fontsize=11, fontweight='bold')
plt.legend()

plt.tight_layout()
plt.show()

save_path = r'results\gain_control_analysis.png'
plt.savefig(save_path, dpi=300)


Gain Increase: 33.5%


## <span style='color:red; font-weight:bold'> What would happen if we 'push' the inhibitory cells?  </span>

In [ ]:
sim.initialize(netParams=netParams, simConfig=simConfig)

# note: if you've done the optogenetic part before, be sure to remove those stim definitions below
# del netParams.stimSourceParams['optogenetic_laser']
# del netParams.stimTargetParams['laser->PV_cells']

# define the stimulus (push to inhibition)
netParams.stimSourceParams['isn_push'] = {
    'type': 'NetStim',
    'rate': 150,     # strong targeted stimulus
    'noise': 0.6,
    'number': 1000,
    'start': 500     # start the push halfway through
}

# target inhibitory PV cells
netParams.stimTargetParams['isn_push->PV'] = {
    'source': 'isn_push',
    'conds': {'pop': 'Inh_PV'}, 
    'weight': 0.1,
    'delay': 1.0,
    'synMech': 'exc' # we are exciting the inhibitory cells
}

# simulation
simConfig.filename = r'results\ISN_experiment_results'
sim.createSimulateAnalyze(netParams=netParams, simConfig=simConfig)


Start time:  2026-02-01 13:20:21.489873

Start time:  2026-02-01 13:20:21.490838




Creating network of 4 cell populations on 1 hosts...: 100%|##########|


  Number of cells on node 0: 100 
  Done; cell creation time = 0.04 s.
Making connections...


  PV->Exc: 100%|##########| Creating synaptic connections for 80/80 postsynaptic cells on node 0 (probabilistic connectivity)
  Exc->PV: 100%|##########| Creating synaptic connections for 10/10 postsynaptic cells on node 0 (probabilistic connectivity)
  SOM->All: 100%|##########| Creating synaptic connections for 90/90 postsynaptic cells on node 0 (probabilistic connectivity)
  Exc->Exc: 100%|##########| Creating synaptic connections for 100/100 postsynaptic cells on node 0 (probabilistic connectivity)


  Number of connections on node 0: 4359 
  Number of synaptic contacts on node 0: 4533 
  Done; cell connection time = 0.66 s.
Adding stims...
  Number of stims on node 0: 110 
  Done; cell stims creation time = 0.01 s.
Recording 4 traces of 1 types on node 0

Running simulation using NEURON for 1000.0 ms...
  Done; run time = 12.47 s; real-time ratio: 0.08.

Gathering data...
  Done; gather time = 0.12 s.

Analyzing...
  Cells: 100
  Connections: 4459 (44.59 per cell)
  Synaptic contacts: 4643 (46.43 per cell)
  Spikes: 2121 (21.21 Hz)
  Simulated time: 1.0 s; 1 workers
  Run time: 12.47 s
Saving output as results\ISN_experiment_results_data.json ... 
Finished saving!
  Done; saving time = 0.29 s.
Preparing spike data...
Plotting raster...
Plotting recorded cell traces ... cell
Plotting connectivity matrix...
Plotting 2D representation of network cell locations and connections...
Plotting firing rate power spectral density (PSD) ...
Plotting spike stats...
  Done; plotting time = 4.13

## Inhibition Stabilized Networks (ISNs) are cortical circuits where strong inhibitory feedback prevents high-level, unstable recurrent excitation from causing runaway activity.

In [13]:
# extract data
spkts = np.array(sim.allSimData['spkt'])
spkids = np.array(sim.allSimData['spkid'])

# windows
pre_isn = [200, 500]
post_isn = [500, 800]

# gid 
pv_gids = [c.gid for c in sim.net.cells if c.tags['pop'] == 'Inh_PV']
exc_gids = [c.gid for c in sim.net.cells if c.tags['pop'] in ['Exc_L5', 'Exc_L23']]

def get_rate(gids, window):
    mask = (spkts >= window[0]) & (spkts < window[1])
    count = np.isin(spkids[mask], gids).sum()
    return count / (len(gids) * (window[1]-window[0])/1000.0)

pv_rate_pre = get_rate(pv_gids, pre_isn)
pv_rate_post = get_rate(pv_gids, post_isn)
exc_rate_pre = get_rate(exc_gids, pre_isn)
exc_rate_post = get_rate(exc_gids, post_isn)


print(f"PV Rate Change: {pv_rate_pre:.2f}Hz -> {pv_rate_post:.2f}Hz")
print(f"Exc Rate Change: {exc_rate_pre:.2f}Hz -> {exc_rate_post:.2f}Hz")

if pv_rate_post < pv_rate_pre:
    print("This circuit is an ISN!")
else:
    print("normal response.")

PV Rate Change: 70.33Hz -> 48.67Hz
Exc Rate Change: 13.00Hz -> 17.17Hz
This circuit is an ISN!


### Because the Exc cells are now firing less, they stop sending Excitation back to the PV cells.